### IBM Data Science Capstone project

### Battle of Neighborhoods: part-1, week-2 (Final)

#### Imporiting required Libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Required Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [4]:
# The code was removed by Watson Studio for sharing.

,State,County,City,RegionID,Name,Geo Point,Geo Shape
0,CA,Sonoma,Santa Rosa,761261,Aston Ave,"38.4244388347, -122.705772408",NaN
1,CA,Sonoma,Santa Rosa,761260,Olive Corby,"38.4296357683, -122.718490047",NaN
2,CA,Sonoma,Santa Rosa,761262,South Park,"38.4282367292, -122.707063534",NaN
3,CA,Sonoma,Santa Rosa,761259,Corby/Beachwood,"38.4228326645, -122.719420258",NaN
4,CA,Sonoma,Santa Rosa,761458,Sunset McMinn,"38.4244068703, -122.729983643",NaN


In [5]:
df_data_0.shape

(33, 7)

### Data Wrangling

In [6]:
# Converting raw data to DataFrame
df = pd.DataFrame(data=df_data_0)

In [7]:
# Extracting the required variables or columns
df = df[["City", "Name", "Geo Point"]]

In [8]:
# Renaming the columns
df= df.rename(columns = {"City":"Borough", "Name":"Neighborhood", "Geo Point":"geocodes"})


In [9]:
# Removing comma from geocodes column
df["geocodes"] = df["geocodes"].str.replace(",","")

In [10]:
# Creating new columns latitude and columns
# df['latitide'] = df['geocodes'].str.split('(', 1).str[0].str.strip()
df['Latitude'] = df['geocodes']
df['Longitude'] = df['geocodes']

In [11]:
# Stripping unwanted characters from latitude and longitude columns
df['Latitude'] = df['Latitude'].str[:13]
df['Longitude'] = df['Longitude'].str[14:28]

In [12]:
df['Latitude'] = df['Latitude'].astype(float)
df['Longitude'] = df['Longitude'].astype(float)

In [13]:
# # Dropping unwanted column - geocodes
df = df.drop(columns=['geocodes'])
df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Santa Rosa,Aston Ave,38.424439,-122.705772
1,Santa Rosa,Olive Corby,38.429636,-122.718490
2,Santa Rosa,South Park,38.428237,-122.707064
3,Santa Rosa,Corby/Beachwood,38.422833,-122.719420
4,Santa Rosa,Sunset McMinn,38.424407,-122.729984


In [15]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
len(df['Borough'].unique()),
df.shape[0]
    )
)

The dataframe has 2 boroughs and 33 neighborhoods.


### Using geopy library to get the latitude and longitude values of Santa Rosa city in Sonoma county.
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent santa_explorer, as shown below.

In [17]:
nom = Nominatim()
n = nom.geocode("Santa Rosa, USA")
Latitude = n.latitude
Longitude = n.longitude
print('The geograpical coordinates of Napa city are {}, {}.'.format(Latitude, Longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  if __name__ == '__main__':


The geograpical coordinates of Napa city are 38.4404925, -122.7141049.


### Creating a map of Santa Rosa with neighborhoods superimposed on top.


In [20]:
# creating map of Santa Rosa using latitude and longitude values
map_santa = folium.Map(location = [Latitude, Longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_santa)

map_santa

Folium is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to
reveal the name of the neighborhood and its respective borough.

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods
in Napa city. So let's slice the original dataframe and create a new dataframe of the Napa data.

In [21]:
napa_data = df[df['Borough'] == 'Napa'].reset_index(drop=True)
napa_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Napa,Alta Heights,38.307360,-122.270429
1,Napa,Beard,38.318796,-122.301167
2,Napa,Bel Aire,38.329107,-122.310260
3,Napa,Browns Valley Central,38.310494,-122.336698
4,Napa,Browns Valley East,38.307756,-122.323485


Getting Geo codes for Napa city

In [22]:
nom = Nominatim()
n = nom.geocode("Napa, USA")
Latitude = n.latitude
Longitude = n.longitude
print('The geograpical coordinates of Napa city are {}, {}.'.format(Latitude, Longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  if __name__ == '__main__':


The geograpical coordinates of Napa city are 38.2971367, -122.2855293.


As we did with all of Santa Rosa, let's visualizat Napa the neighborhoods in it.

In [24]:
map_napa = folium.Map(location=[Latitude, Longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(napa_data['Latitude'], napa_data['Longitude'], napa_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_napa)

map_napa

### Define Foursquare Credentials and Version
Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [25]:
CLIENT_ID = 'SR2LDZKIXARRVINIY4RYR2XTETJT0IAIDSXPPLPSUTXLVUEV' # your Foursquare ID
CLIENT_SECRET = '0533WJN2IE3CK2QREGNDBPRLXLDBG1URBRWIF0PUJ5GZEUZS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SR2LDZKIXARRVINIY4RYR2XTETJT0IAIDSXPPLPSUTXLVUEV
CLIENT_SECRET:0533WJN2IE3CK2QREGNDBPRLXLDBG1URBRWIF0PUJ5GZEUZS


#### Let's explore the first neighborhood in our dataframe.
Get the neighborhood's name.

In [27]:
napa_data.loc[0, 'Neighborhood']

'Alta Heights'

In [28]:
# Get the neighborhood's latitude and longitude values.
neighborhood_latitude = napa_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = napa_data.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = napa_data.loc[0, 'Neighborhood'] # neighborhood name
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name,
neighborhood_latitude,
neighborhood_longitude))

Latitude and longitude values of Alta Heights are 38.3073602382, -122.270428502.


#### Now, let's get the top 100 venues that are in Napa city within a radius of 1000 meters.
First, let's create the GET request URL. Name your URL url.

In [29]:
# type your answer here
radius = 1000 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    neighborhood_latitude,
    neighborhood_longitude,
    radius,
    LIMIT)

url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=SR2LDZKIXARRVINIY4RYR2XTETJT0IAIDSXPPLPSUTXLVUEV&client_secret=0533WJN2IE3CK2QREGNDBPRLXLDBG1URBRWIF0PUJ5GZEUZS&v=20180605&ll=38.3073602382,-122.270428502&radius=1000&limit=100'

In [30]:
results = requests.get(url).json()

From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we
proceed, let's borrow the get_category_type function from the Foursquare lab.

In [31]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [32]:
# Now we are ready to clean the json and structure it into a pandas dataframe.
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Big D Burgers,Burger Joint,38.301350,-122.276545
1,Napa Tire Inc,Automotive Shop,38.310177,-122.276994
2,El Guadalajara,Mexican Restaurant,38.305510,-122.277364
3,RiverPointe Napa Valley Resort,Resort,38.311415,-122.279088
4,Compadres Rio Grille,Mexican Restaurant,38.309173,-122.279436


And how many venues were returned by Foursquare?

In [33]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

16 venues were returned by Foursquare.


### Explore Neighborhoods in Napa

Let's create a function to repeat the same process to all the neighborhoods in Napa

In [34]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID,
                CLIENT_SECRET,
                VERSION,
                lat,
                lng,
                radius,
                LIMIT)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
        name,
        lat,
        lng,
        v['venue']['name'],
        v['venue']['location']['lat'],
        v['venue']['location']['lng'],
        v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
        'Neighborhood Latitude',
        'Neighborhood Longitude',
        'Venue',
        'Venue Latitude',
        'Venue Longitude',
        'Venue Category']

    return(nearby_venues)
        

Now write the code to run the above function on each neighborhood and create a new dataframe called napa_venues.

In [35]:
napa_venues = getNearbyVenues(names=napa_data['Neighborhood'],
latitudes=napa_data['Latitude'],
longitudes=napa_data['Longitude']
)

Alta Heights
Beard
Bel Aire
Browns Valley Central
Browns Valley East
Browns Valley South
Browns Valley West
Central
Coventry Manor
Downtown
Fuller Park
Linda Vista
Mcpherson
Napa Abajo
Pueblo Park
Riverpark
Sheveland Ranch
Shurtleff
Soscol Gateway North
Soscol Gateway South
Springwood Estates
Stanly Ranch
Stonehouse
Terrace
Vineyard Estates
Vintage
Von Uhlit Ranch
Westwood


In [36]:
# check the size of the resulting dataframe
print(napa_venues.shape)
napa_venues.head()

(883, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Alta Heights,38.30736,-122.270429,Big D Burgers,38.301350,-122.276545,Burger Joint
1,Alta Heights,38.30736,-122.270429,Napa Tire Inc,38.310177,-122.276994,Automotive Shop
2,Alta Heights,38.30736,-122.270429,El Guadalajara,38.305510,-122.277364,Mexican Restaurant
3,Alta Heights,38.30736,-122.270429,RiverPointe Napa Valley Resort,38.311415,-122.279088,Resort
4,Alta Heights,38.30736,-122.270429,Compadres Rio Grille,38.309173,-122.279436,Mexican Restaurant


In [122]:
# check how many venues were returned for each neighborhood
napa_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Alta Heights,1,1,1,1,1,1
Beard,45,45,45,45,45,45
Bel Aire,3,3,3,3,3,3
Browns Valley Central,2,2,2,2,2,2
Browns Valley South,3,3,3,3,3,3
Browns Valley West,3,3,3,3,3,3
Central,4,4,4,4,4,4
Coventry Manor,1,1,1,1,1,1
Downtown,100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues

In [37]:
print('There are {} uniques categories.'.format(len(napa_venues['Venue Category'].unique())))

There are 165 uniques categories.


### Analyze Each Neighborhood

In [38]:
# one hot encoding
napa_onehot = pd.get_dummies(napa_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
napa_onehot['Neighborhood'] = napa_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [napa_onehot.columns[-1]] + list(napa_onehot.columns[:-1])
napa_onehot = napa_onehot[fixed_columns]

napa_onehot.head()

,Neighborhood,ATM,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Bed & Breakfast,Beer Bar,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Station,Business Service,Café,Campground,Candy Store,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,College Cafeteria,College Gym,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Credit Union,Cupcake Shop,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Donut Shop,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Flower Shop,Food & Drink Shop,Food Truck,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hardware Store,Himalayan Restaurant,Home Service,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Insurance Office,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Lake,Lingerie Store,Liquor Store,Lounge,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Mobile Phone Shop,Motel,Movie Theater,Music Store,Music Venue,Nail Salon,Noodle House,Opera House,Optical Shop,Other Repair Shop,Outlet Mall,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Pool,Pool Hall,Print Shop,Ramen Restaurant,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Rock Club,Roof Deck,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Tour Provider,Tourist Information Center,Track,Trail,Train,Video Store,Vietnamese Restaurant,Vineyard,Waterfront,Wine Bar,Wine Shop,Winery,Women's Store
0,Alta Heights,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Alta Heights,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Alta Heights,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Alta Heights,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Alta Heights,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [39]:
napa_onehot.shape

(883, 166)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [40]:
napa_grouped = napa_onehot.groupby('Neighborhood').mean().reset_index()
napa_grouped

,Neighborhood,ATM,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Bed & Breakfast,Beer Bar,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Station,Business Service,Café,Campground,Candy Store,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,College Cafeteria,College Gym,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Credit Union,Cupcake Shop,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Donut Shop,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Flower Shop,Food & Drink Shop,Food Truck,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hardware Store,Himalayan Restaurant,Home Service,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Insurance Office,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Lake,Lingerie Store,Liquor Store,Lounge,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Mobile Phone Shop,Motel,Movie Theater,Music Store,Music Venue,Nail Salon,Noodle House,Opera House,Optical Shop,Other Repair Shop,Outlet Mall,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Pool,Pool Hall,Print Shop,Ramen Restaurant,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Rock Club,Roof Deck,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Tour Provider,Tourist Information Center,Track,Trail,Train,Video Store,Vietnamese Restaurant,Vineyard,Waterfront,Wine Bar,Wine Shop,Winery,Women's Store
0,Alta Heights,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.0625,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0625,0.000000,0.00,0.000000,0.062500,0.0,0.0,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.187500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.00,0.000000,0.000000,0.062500,0.0625,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0625,0.062500,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
1,Beard,0.012048,0.000000,0.024096,0.00,0.00,0.000000,0.0000,0.000000,0.012048,0.012048,0.012048,0.024096,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012048,0.000000,0.000000,0.000000,0.000000,0.000000,0.024096,0.000000,0.000000,0.000000,0.000000,0.012048,0.000000,0.0000,0.012048,0.00,0.

In [41]:
napa_grouped.shape

(28, 166)

#### Let's print each neighborhood along with the top 5 most common venues

In [42]:
num_top_venues = 5

for hood in napa_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = napa_grouped[napa_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alta Heights----
                venue  freq
0  Mexican Restaurant  0.19
1        Burger Joint  0.06
2               Trail  0.06
3          Restaurant  0.06
4   Convenience Store  0.06


----Beard----
                  venue  freq
0           Coffee Shop  0.06
1  Fast Food Restaurant  0.06
2    Mexican Restaurant  0.05
3         Grocery Store  0.04
4           Pizza Place  0.04


----Bel Aire----
                  venue  freq
0                 Hotel  0.06
1    Mexican Restaurant  0.06
2  Fast Food Restaurant  0.06
3           Coffee Shop  0.06
4   American Restaurant  0.06


----Browns Valley Central----
            venue  freq
0  Massage Studio  0.14
1    Home Service  0.14
2    Dessert Shop  0.14
3            Park  0.14
4     Coffee Shop  0.14


----Browns Valley East----
                        venue  freq
0                        Park  0.25
1                Dessert Shop  0.12
2  Construction & Landscaping  0.12
3                      Garden  0.12
4                 Coffee Shop  

#### Let's put that into a pandas dataframe
First, let's write a function to sort the venues in descending order.

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)

    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [44]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

        # create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = napa_grouped['Neighborhood']

for ind in np.arange(napa_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(napa_grouped.iloc[ind, :], num_top_venues)
    
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alta Heights,Mexican Restaurant,Restaurant,Trail,Child Care Service,Coffee Shop,Gas Station,Resort,Track,Burger Joint,Automotive Shop
1,Beard,Fast Food Restaurant,Coffee Shop,Mexican Restaurant,Grocery Store,Pizza Place,Steakhouse,Cosmetics Shop,Pet Store,Mobile Phone Shop,Liquor Store
2,Bel Aire,Hotel,American Restaurant,Fast Food Restaurant,Mexican Restaurant,Coffee Shop,Restaurant,Furniture / Home Store,Italian Restaurant,Mobile Phone Shop,Pharmacy
3,Browns Valley Central,Home Service,Grocery Store,Massage Studio,Gastropub,Coffee Shop,Park,Dessert Shop,Department Store,Diner,Discount Store
4,Browns Valley East,Park,Construction & Landscaping,Grocery Store,Coffee Shop,Dessert Shop,Garden,Gastropub,Women's Store,Donut Shop,Fast Food Restaurant


### Cluster Neighborhoods
Run k-means to cluster the neighborhood into 5 clusters.

In [45]:
# set number of clusters
kclusters = 5

napa_grouped_clustering = napa_grouped.drop('Neighborhood', 1)
# run k-means clustering

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(napa_grouped_clustering)
# check cluster labels generated for each row in the dataframe

kmeans.labels_[0:10]

array([2, 2, 2, 0, 0, 0, 4, 2, 2, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [46]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

napa_merged = napa_data

# merge napa_grouped with napa_data to add latitude/longitude for each neighborhood
napa_merged = napa_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on ='Neighborhood')

napa_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Napa,Alta Heights,38.307360,-122.270429,2,Mexican Restaurant,Restaurant,Trail,Child Care Service,Coffee Shop,Gas Station,Resort,Track,Burger Joint,Automotive Shop
1,Napa,Beard,38.318796,-122.301167,2,Fast Food Restaurant,Coffee Shop,Mexican Restaurant,Grocery Store,Pizza Place,Steakhouse,Cosmetics Shop,Pet Store,Mobile Phone Shop,Liquor Store
2,Napa,Bel Aire,38.329107,-122.310260,2,Hotel,American Restaurant,Fast Food Restaurant,Mexican Restaurant,Coffee Shop,Restaurant,Furniture / Home Store,Italian Restaurant,Mobile Phone Shop,Pharmacy
3,Napa,Browns Valley Central,38.310494,-122.336698,0,Home Service,Grocery Store,Massage Studio,Gastropub,Coffee Shop,Park,Dessert Shop,Department Store,Diner,Discount Store
4,Napa,Browns Valley East,38.307756,-122.323485,0,Park,Construction & Landscaping,Grocery Store,Coffee Shop,Dessert Shop,Garden,Gastropub,Women's Store,Donut Shop,Fast Food Restaurant


Finally, let's visualize the resulting clusters

In [48]:
# create map
map_clusters = folium.Map(location=[Latitude, Longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []

for lat, lon, poi, cluster in zip(napa_merged['Latitude'], napa_merged['Longitude'], 
                                  napa_merged['Neighborhood'], napa_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
    radius=5,
    popup=label,
    color=rainbow[cluster-1],
    fill=True,
    fill_color=rainbow[cluster-1],
    fill_opacity=0.7).add_to(map_clusters)

map_clusters

### Examine Clusters
Now, you can examine each cluster and determine the discriminating venue categories that distinguish eachcluster.
#### Cluster-1

In [49]:
napa_merged.loc[napa_merged['Cluster Labels'] == 0, napa_merged.columns[[1]+ list(range(5, napa_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Browns Valley Central,Home Service,Grocery Store,Massage Studio,Gastropub,Coffee Shop,Park,Dessert Shop,Department Store,Diner,Discount Store
4,Browns Valley East,Park,Construction & Landscaping,Grocery Store,Coffee Shop,Dessert Shop,Garden,Gastropub,Women's Store,Donut Shop,Fast Food Restaurant
5,Browns Valley South,Park,Lake,American Restaurant,Massage Studio,Scenic Lookout,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
24,Vineyard Estates,Food Truck,Park,Bus Station,Baseball Stadium,Grocery Store,Mexican Restaurant,Hotel Pool,Trail,Italian Restaurant,Tour Provider


#### Cluster-2

In [50]:
napa_merged.loc[napa_merged['Cluster Labels'] == 1, napa_merged.columns[[1]+ list(range(5, napa_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Stanly Ranch,Cosmetics Shop,Farm,Food Truck,Flower Shop,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Donut Shop


#### Cluster- 3

In [51]:
napa_merged.loc[napa_merged['Cluster Labels'] == 2, napa_merged.columns[[1]+ list(range(5, napa_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alta Heights,Mexican Restaurant,Restaurant,Trail,Child Care Service,Coffee Shop,Gas Station,Resort,Track,Burger Joint,Automotive Shop
1,Beard,Fast Food Restaurant,Coffee Shop,Mexican Restaurant,Grocery Store,Pizza Place,Steakhouse,Cosmetics Shop,Pet Store,Mobile Phone Shop,Liquor Store
2,Bel Aire,Hotel,American Restaurant,Fast Food Restaurant,Mexican Restaurant,Coffee Shop,Restaurant,Furniture / Home Store,Italian Restaurant,Mobile Phone Shop,Pharmacy
7,Central,Hotel,Wine Bar,American Restaurant,Mexican Restaurant,Bed & Breakfast,Italian Restaurant,BBQ Joint,Pizza Place,Spa,Restaurant
8,Coventry Manor,Construction & Landscaping,Playground,Arts & Crafts Store,Café,Food Truck,Donut Shop,Women's Store,Falafel Restaurant,Flower Shop,Financial or Legal Service
9,Downtown,American Restaurant,Mexican Restaurant,Wine Bar,Bed & Breakfast,Hotel,Italian Restaurant,Sushi Restaurant,Bakery,Burger Joint,Spa
10,Fuller Park,Clothing Store,Hotel,Bed & Breakfast,Sporting Goods Shop,Breakfast Spot,Cosmetics Shop,Mexican Restaurant,Playground,Shoe Store,Women's Store
11,Linda Vista,Hotel,Convenience Store,Food Truck,Lounge,Burger Joint,American Restaurant,Ice Cream Shop,Brazilian Restaurant,Park,Coffee Shop
12,Mcpherson,Pizza Place,Coffee Shop,Sandwich Place,Taco Place,Print Shop,Fast Food Restaurant,BBQ Joint,Mexican Restaurant,Salon / Barbershop,Pet Store
13,Napa Abajo,Breakfast Spot,Bed & Breakfast,Hotel,Massage Studio,Rental Car Location,Pizza Place,Pharmacy,Shopping Mall,Mexican Restaurant,Arts & Crafts Store


#### Cluster- 4

In [52]:
napa_merged.loc[napa_merged['Cluster Labels'] == 3, napa_merged.columns[[1]+ list(range(5, napa_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Westwood,Clothing Store,Shoe Store,Sporting Goods Shop,Accessories Store,American Restaurant,Coffee Shop,Mexican Restaurant,Burger Joint,Scenic Lookout,Food & Drink Shop


#### Cluster- 5

In [53]:
napa_merged.loc[napa_merged['Cluster Labels'] == 4, napa_merged.columns[[1]+ list(range(5, napa_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Browns Valley West,Vineyard,Hotel,Seafood Restaurant,Wine Bar,Scenic Lookout,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space


## Conclusion

Based on the Clusters above, the company should start new themed restaurant in the Browns Valley West of cluster- 5 because it has has farm and farmers market for the supply of ingredients and other raw materials.
It has also Scenic Layout and Vineyard that attracts tourists & other visitors, and is suitable to open a new themed restaurant

Second option would be opening of new restaurant in Browns Valley South of Cluster- 1 as this has also park, lake and farmers market.
And last option would be in Stanly Ranch in cluster-2